## Standard imports

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import re
import itertools
from pandas.io.json import json_normalize
from datetime import datetime

### Loading the data and getting the right types

In [2]:
credits = pd.read_csv('raw data/credits.csv')
keywords = pd.read_csv('raw data/keywords.csv')
rating = pd.read_csv('raw data/ratings.csv')

credits.shape, keywords.shape, rating.shape

((45476, 3), (46419, 2), (26024289, 4))

In [3]:
movieData = pd.read_csv('raw data/movies_metadata.csv', low_memory=False)
patternDel = "([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))"
filter = movieData['id'].str.contains(patternDel)
movieData = movieData[~filter]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
movieData.overview = movieData.overview.apply(str)
movieData.spoken_languages = movieData.spoken_languages.apply(str)

In [5]:
#converting types and renaming columns
movieData[["id"]] = movieData[["id"]].apply(pd.to_numeric)
movieData[["popularity"]] = movieData[["popularity"]].apply(pd.to_numeric)
movieData[["budget"]] = movieData[["budget"]].apply(pd.to_numeric)
movieData = movieData.rename(columns={'id': 'movie_id'})
movieData.dtypes
movieData.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,movie_id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
37410,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,45205,tt0884819,ja,ウール100％,A drama of two aging women who live a solitary...,...,2006-01-01,0.0,99.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Wool 100%,False,5.8,3.0
24568,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,34995,tt0340919,en,Puerto Vallarta Squeeze,An American government hit man on the run make...,...,2003-02-26,0.0,117.0,[],Released,NaN,Puerto Vallarta Squeeze,False,4.6,4.0
41102,False,NaN,0,"[{'id': 10752, 'name': 'War'}, {'id': 16, 'nam...",NaN,413770,tt1725969,en,Ethel & Ernest,"This hand drawn animated film, based on the aw...",...,2016-10-28,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A true story.,Ethel & Ernest,False,7.8,20.0
32403,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,117550,tt0215837,en,"Helga, la louve de Stilberg","Helga, a woman who runs a strict prison camp, ...",...,1977-10-20,0.0,93.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,"Helga, She Wolf of Spilberg",False,4.0,1.0
9775,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,76804,tt0017739,en,The Cat and the Canary,Rich old Cyrus West's relatives are waiting fo...,...,1927-09-09,0.0,82.0,[],Released,NaN,The Cat and the Canary,False,7.0,12.0


In [6]:
#removing the rows that are almost completley nan
movieData = movieData[np.isfinite(movieData['revenue'])]

In [7]:
def unpack(movie_id, details):
    elements = [element for element in eval(details)]
    
    for element in elements:
        element['movie_id'] = movie_id
        
    return elements

In [8]:
cast_with_movie_id = credits.apply(lambda row: unpack(row['id'], row['cast']), axis=1)
crew_with_movie_id = credits.apply(lambda row: unpack(row['id'], row['crew']), axis=1)
keywords_with_movie_id = keywords.apply(lambda row: unpack(row['id'], row['keywords']), axis=1)
languages_changed = movieData.apply(lambda row: unpack(row['movie_id'], row['spoken_languages']), axis=1)

In [9]:
cast_list = itertools.chain.from_iterable(cast_with_movie_id.values)
crew_list = itertools.chain.from_iterable(crew_with_movie_id.values)
keywords_list = itertools.chain.from_iterable(keywords_with_movie_id.values)
language_list = itertools.chain.from_iterable(languages_changed.values)

cast_df = pd.DataFrame(list(cast_list))
crew_df = pd.DataFrame(list(crew_list))
keywords_df = pd.DataFrame(list(keywords_list))
language_df = pd.DataFrame(list(language_list))

cast_df.shape, crew_df.shape, keywords_df.shape

((562474, 9), (464314, 8), (158680, 3))

In [10]:
cast_df = cast_df.drop(['cast_id','character','credit_id','gender','id','order','profile_path'], axis=1)

#gapminder_ocean.drop(['pop', 'gdpPercap', 'continent'], axis=1)

In [11]:
language_df.head(5)

,iso_639_1,movie_id,name
0,en,862,English
1,en,8844,English
2,fr,8844,Français
3,en,15602,English
4,en,31357,English


In [12]:
languages = language_df.groupby('movie_id')['name'].apply(','.join).reset_index()
languages = languages.rename(columns={'name': 'languages'})
movieData_with_languages = pd.merge(movieData, languages[['movie_id', 'languages']], on='movie_id', how='left')

In [13]:
movieData_with_languages = movieData_with_languages.drop(['spoken_languages','belongs_to_collection','adult','original_language','homepage','tagline'], axis = 1)
movieData_with_languages.head(5)

,budget,genres,movie_id,imdb_id,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,title,video,vote_average,vote_count,languages
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0,English
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0,"English,Français"
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0,English
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0,English
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0,English


In [14]:
#function to remove all the unwanted characters in the genres columns
movieData_with_languages['genres'] = movieData_with_languages['genres'].apply(lambda x: re.findall('(?<!^)(?<!\. )[A-Z][a-z]+',x))
movieData_with_languages.sample(1)

,budget,genres,movie_id,imdb_id,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,title,video,vote_average,vote_count,languages
26156,750000,"[Comedy, Music, Mystery]",151509,tt2526856,Trapped in the Closet: Chapters 23-33,"The long awaited sequel to R. Kelly's ""Trapped...",0.433321,/bQcXy40C4xO8CFGyIdvkHU3Ek6I.jpg,"[{'name': 'Relevant', 'id': 83440}, {'name': '...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-11-23,0.0,45.0,Released,Trapped in the Closet: Chapters 23-33,False,6.5,4.0,English


In [15]:
#taking a list in the genres column and turning it into multiple rows in the dataframe
movieData_with_languages_and_genres = movieData_with_languages.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
movieData_with_languages_and_genres.name = 'genre'
movieData_with_languages_and_genres = movieData_with_languages.drop('genres', axis=1).join(movieData_with_languages_and_genres).reset_index()

In [16]:
flag = False
if(flag==True):
    movieData_with_languages_and_genres.to_csv('movieData_with_languages_and_genres.csv', sep=',', encoding='utf-8', index=False)

### Function to unpack the json strings
This function puts the values of the key in each dictionary into its own column.

In [17]:
keywords_df = keywords_df.groupby('movie_id')['name'].apply(','.join).reset_index()
keywords_with_movie_title = pd.merge(keywords_df, movieData[['movie_id', 'title']], on='movie_id', how='left')
keywords_with_movie_title = keywords_with_movie_title.rename(columns={'name': 'keywords'})
keywords_with_movie_title.head(5)

,movie_id,keywords,title
0,2,"underdog,prison,factory worker,prisoner,helsin...",Ariel
1,3,"salesclerk,helsinki,garbage,independent film",Shadows in Paradise
2,5,"hotel,new year's eve,witch,bet,hotel room,sper...",Four Rooms
3,6,"chicago,drug dealer,boxing match,escape,one night",Judgment Night
4,11,"android,galaxy,hermit,death star,lightsaber,je...",Star Wars


In [18]:
directors_df = crew_df[crew_df['job'] == 'Director']

In [19]:
directors_df = directors_df.drop(['credit_id', 'profile_path', 'gender'], axis = 1)
directors_df.shape

(49048, 5)

In [20]:
directors_with_movies = pd.merge(movieData_with_languages, directors_df[['movie_id','name']], on ='movie_id', how='left' )


if(flag==True):
    directors_with_movies.to_csv('directors_with_movies.csv', sep=',', encoding='utf-8', index=False)

In [21]:
directors_with_movies.sample(1)

,budget,genres,movie_id,imdb_id,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,title,video,vote_average,vote_count,languages,name
49158,0,"[Drama, Romance]",52039,tt0760166,Don't Let Me Drown,In a post-September 11th world overflowing wit...,0.371407,/zXrVmD8OaBzEc3Uybf5S9DdTuT1.jpg,"[{'name': 'Parts and Labor', 'id': 6531}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-01-01,0.0,105.0,Released,Don't Let Me Drown,False,6.0,1.0,English,Cruz Angeles


In [22]:
def unpack_movieData(budget, genres, movie_id, details, release_date, revenue, runtime, status, title, video, vote_average, vote_count, languages):
    elements = [element for element in eval(details)]
    
    for element in elements:
        element['movie_id'] = movie_id
        element['genres'] = genres
        element['budget'] = budget
        element['release_date'] = release_date
        element['revenue'] = revenue
        element['runtime'] = runtime
        element['status'] = status
        element['title'] = title
        element['video'] = video
        element['vote_average'] = vote_average
        element['vote_count'] = vote_count
        element['languages'] = languages
        
       
    
    return elements

production_companies_unpacked = movieData_with_languages.apply(lambda row: unpack_movieData(row['budget'],row['genres'],row['movie_id'],row['production_companies'],row['release_date'],row['revenue'],row['runtime'],row['status'],row['title'],row['video'],row['vote_average'],row['vote_count'],row['languages']), axis=1)


In [23]:
production_company_list = itertools.chain.from_iterable(production_companies_unpacked.values)
production_company_df = pd.DataFrame(list(production_company_list))
movieData_clean = production_company_df.rename(columns={'name': 'production_company'})

In [24]:
movieData_shortened = movieData_with_languages_and_genres.drop(['index','budget',
                                                   'imdb_id','original_title','overview','popularity', 'poster_path',
                                                   'production_companies','production_countries','release_date','revenue',
                                                   'runtime','status','video','languages'], axis = 1)

In [25]:
movieData_clean.head(10)

,budget,genres,id,languages,movie_id,production_company,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,30000000,"[Animation, Comedy, Family]",3,English,862,Pixar Animation Studios,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,65000000,"[Adventure, Fantasy, Family]",559,"English,Français",8844,TriStar Pictures,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,65000000,"[Adventure, Fantasy, Family]",2550,"English,Français",8844,Teitler Film,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
3,65000000,"[Adventure, Fantasy, Family]",10201,"English,Français",8844,Interscope Communications,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
4,0,"[Romance, Comedy]",6194,English,15602,Warner Bros.,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
5,0,"[Romance, Comedy]",19464,English,15602,Lancaster Gate,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
6,16000000,"[Comedy, Drama, Romance]",306,English,31357,Twentieth Century Fox Film Corporation,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
7,0,[Comedy],5842,English,11862,Sandollar Productions,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0
8,0,[Comedy],9195,English,11862,Touchstone Pictures,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0
9,60000000,"[Action, Crime, Drama, Thriller]",508,"English,Español",949,Regency Enterprises,1995-12-15,187436818.0,170.0,Released,Heat,False,7.7,1886.0


In [26]:
movieData_clean.dtypes

budget                  int64
genres                 object
id                      int64
languages              object
movie_id                int64
production_company     object
release_date           object
revenue               float64
runtime               float64
status                 object
title                  object
video                    bool
vote_average          float64
vote_count            float64
dtype: object

In [27]:
movieData_with_languages['revenue'] = movieData_with_languages['revenue'].astype('int')
movieData_with_languages['profit/loss%'] = (((movieData_with_languages['revenue']-movieData_with_languages['budget'])/movieData_with_languages['budget'])*100)

In [28]:
movieData_with_languages = movieData_with_languages.replace([np.inf, -np.inf], np.nan)
movieData_with_languages = movieData_with_languages[np.isfinite(movieData_with_languages['profit/loss%'])]

In [31]:
movieData_with_languages.sample(4)

,budget,genres,movie_id,imdb_id,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,title,video,vote_average,vote_count,languages,profit/loss%
14211,20000000,[Documentary],22074,tt1232207,Capitalism: A Love Story,Michael Moore's Capitalism: A Love Story comes...,6.032166,/zJBkIPakBXdu3cEgGqKCPHUNXVE.jpg,"[{'name': 'The Weinstein Company', 'id': 308},...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-09-06,17436509,120.0,Released,Capitalism: A Love Story,False,7.0,168.0,"English,Polski",-12.817455
14374,65000000,"[Animation, Action, Family, Science, Fiction]",16577,tt0375568,Astro Boy,"Set in futuristic Metro City, Astro Boy is abo...",13.293466,/4kQczIhUFTnDWwG6HKsgCxLoi6.jpg,"[{'name': 'The Weinstein Company', 'id': 308},...","[{'iso_3166_1': 'HK', 'name': 'Hong Kong'}, {'...",2009-10-15,44091067,94.0,Released,Astro Boy,False,6.1,420.0,English,-32.167589
2247,1300000,[Drama],309,tt0154420,Festen,A grandiose party to celebrate a sixtieth birt...,6.064954,/b0YDV5j53Y5xRscI14UieetvNLB.jpg,"[{'name': 'Nordic Film', 'id': 156}, {'name': ...","[{'iso_3166_1': 'DK', 'name': 'Denmark'}, {'is...",1998-05-20,0,105.0,Released,The Celebration,False,7.7,213.0,"Dansk,Deutsch,English",-100.000000
10817,55000000,"[Action, Adventure, Crime, Thriller]",2207,tt0450232,16 Blocks,An aging cop is assigned the ordinary task of ...,11.887779,/wpKBLDjNRddmH6N32ni4bASogt8.jpg,"[{'name': ""Donners' Company"", 'id': 431}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2006-03-01,65664721,105.0,Released,16 Blocks,False,6.2,675.0,English,19.390402


In [38]:
movieData_with_languages.shape, movieData_with_languages.dtypes

((8890, 20), budget                    int64
 genres                   object
 movie_id                  int64
 imdb_id                  object
 original_title           object
 overview                 object
 popularity              float64
 poster_path              object
 production_companies     object
 production_countries     object
 release_date             object
 revenue                   int64
 runtime                 float64
 status                   object
 title                    object
 video                      bool
 vote_average            float64
 vote_count              float64
 languages                object
 profit/loss%            float64
 dtype: object)

In [39]:
#saving to csv, if the flag is set to true it will download 
flag = False

if(flag==True):
              movieData_with_languages.to_csv('movieData_with_languages.csv', sep=',', encoding='utf-8', index=False)
              cast_df.to_csv('cast.csv', sep=',', encoding='utf-8', index=False)

In [44]:
actors_with_movies = pd.merge(movieData_with_languages, cast_df[['movie_id','name']], on ='movie_id', how='left' )

flag = False
if(flag==True):
              actors_with_movies.to_csv('actors_with_movies.csv', sep=',', encoding='utf-8', index=False)